# TODO:

1. Verify that the merge result is as expected.  I noticed that when I checked the length of the EPA for a 6-month period, the length was longer than the post-merge 6-month period.  This implies that the EIA data doesn't have some facilities that are listed in EPA.  I haven't verified this though.

2. NaN values also need to be replaced with 0's.

3. Numbers are being stored in scientific notation

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import re
import cPickle as pickle
import gzip
import seaborn as sns

/Users/Home/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Loading the EIA Data, the path may need to be updated...
This will take a few minutes to run.

In [2]:
#Iterate through the directory to find all the files to import
#Modified so that it also works on macs
path = os.path.join('EIA Data', '923-No_Header')
full_path = os.path.join(path, '*.*')


eiaNames = os.listdir(path)

#Rename the keys for easier merging later
fileNameMap = {'EIA923 SCHEDULES 2_3_4_5 Final 2010.xls':2010,
                'EIA923 SCHEDULES 2_3_4_5 M Final 2009 REVISED 05252011.XLS':2009,
                'eia923December2008.xls':2008,
                'EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx':2011,
                'EIA923_Schedules_2_3_4_5_2012_Final_Release_12.04.2013.xlsx':2012,
                'EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx':2013,
                'EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx':2014,
                'EIA923_Schedules_2_3_4_5_M_12_2015_Final.xlsx':2015,
                'f906920_2007.xls':2007}

#Load the files into data frames, one df per file
eiaDict = {fileNameMap[fn]:pd.read_excel(os.path.join(path, fn)) for fn in eiaNames}
eiaDict = {key:val[val["NERC Region"] == "TRE"] for key, val in eiaDict.iteritems()}

The excel documents have different column names so we need to standardize them all

In [3]:
#Dict of values to replace to standardize column names across all dataframes
monthDict = {"JANUARY":"JAN",
           "FEBRUARY":"FEB",
           "MARCH":"MAR",
           "APRIL":"APR",
           "MAY":"MAY",
           "JUNE":"JUN",
           "JULY":"JUL",
           "AUGUST":"AUG",
           "SEPTEMBER":"SEP",
           "OCTOBER":"OCT",
           "NOVEMBER":"NOV",
           "DECEMBER":"DEC"}
           
replaceDict = {"ELECTRIC":"ELEC",
               "&":"AND",
               "I.D.":"ID",
               "MMBTUPER":"MMBTU_PER"}
               
#Add "MMBTUMON" : "MMBTU_MON" to be replaced
for month in monthDict.values():
    replaceDict["MMBTU"+month] = "MMBTU_" + month

#Replace the column name
def rename(col):
    for old, new in monthDict.iteritems():
        col = col.replace(old, new)
        
    for old, new in replaceDict.iteritems():
        col = col.replace(old, new)
        
    col = col.replace("MMBTUS", "MMBTU")
    return col
    
#Iterate through each column name of each dataframe to standardize
for key, df in eiaDict.iteritems():
    colNames = [name.replace("\n", "_").replace(" ", "_").strip().upper() for name in df.columns]
    colNames = [rename(col) for col in colNames]
    eiaDict[key].columns = colNames

Define which columns we need to sum, and which columns don't need to be summed, but we still need to keep.

Note: If we don't care about monthly stuff we can delete the second block of code.

In [4]:
#Define the columns that are necessary but are not summable
allCols = eiaDict[fileNameMap.values()[0]].columns
nonSumCols = ["PLANT_ID", "PLANT_NAME", "YEAR"]

#Define the columns that contain the year's totals (Used to calc fuel type %)
yearCols = ["TOTAL_FUEL_CONSUMPTION_QUANTITY", "ELEC_FUEL_CONSUMPTION_QUANTITY",
            "TOTAL_FUEL_CONSUMPTION_MMBTU", "ELEC_FUEL_CONSUMPTION_MMBTU",
            "NET_GENERATION_(MEGAWATTHOURS)"]


#Define the columns that are necessary and summable
sumCols = []
sumCols.extend(yearCols)
# regex = re.compile(r"^ELEC_QUANTITY_.*")
# sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^MMBTU_PER_UNIT_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^TOT_MMBTU_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^ELEC_MMBTUS_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^NETGEN_.*")
sumCols.extend([col for col in allCols if regex.search(col)])

Get a list of all the different fuel type codes.  If we don't care about all of them, then just hardcode the list

In [5]:
fuelTypes = []
fuelTypes.extend([fuelType for df in eiaDict.values() for fuelType in df["REPORTED_FUEL_TYPE_CODE"].tolist()])
fuelTypes = set(fuelTypes)

In [6]:
fuelTypes

{u'AB',
 u'BIT',
 u'BLQ',
 u'DFO',
 u'JF',
 u'LFG',
 u'LIG',
 u'MWH',
 u'NG',
 u'NUC',
 u'OBG',
 u'OBL',
 u'OBS',
 u'OG',
 u'OTH',
 u'PC',
 u'PUR',
 u'RFO',
 u'SC',
 u'SUB',
 u'SUN',
 u'WAT',
 u'WDS',
 u'WH',
 u'WND',
 u'WO'}

3 parts to aggregate by facility, and to calculate the % of each type of fuel.  This will take a few minutes to run.

The end result is aggEIADict.

In [7]:
#Actually calculate the % type for each facility grouping
def calcPerc(group, aggGroup, fuelType, col):
    #Check to see if the facility has a record for the fuel type, and if the total column > 0
    if len(group[group["REPORTED_FUEL_TYPE_CODE"] == fuelType]) > 0 and aggGroup[col] > 0:
        #summing fuel type because a facility may have multiple plants with the same fuel type        
        return float((group[group["REPORTED_FUEL_TYPE_CODE"] == fuelType][col]).sum())/aggGroup[col] 
    else:
        return 0

#Perform the aggregation on facility level
def aggAndCalcPerc(group):
    aggGroup = group.iloc[0][nonSumCols] #Get the non-agg columns
    aggGroup = aggGroup.append(group[sumCols].sum())   #Aggregate the agg columns and append to non-agg
    percCols = {col + " %" + fuelType:calcPerc(group, aggGroup, fuelType, col) for col in yearCols for fuelType in fuelTypes}
    aggGroup = aggGroup.append(pd.Series(percCols))
    return aggGroup    

#Iterate through each dataframe to perform aggregation by facility
aggEIADict = dict()
for key, df in eiaDict.iteritems():
    gb = df.groupby(by="PLANT_ID")
    #aggGroup will be a list of panda series, each series representing a facility
    aggGroup = [aggAndCalcPerc(gb.get_group(group)) for group in gb.groups]
    aggEIADict[key] = pd.DataFrame(aggGroup)

### Column order doesn't match in all years

In [8]:
aggEIADict[2007].head()

,PLANT_ID,PLANT_NAME,YEAR,TOTAL_FUEL_CONSUMPTION_QUANTITY,ELEC_FUEL_CONSUMPTION_QUANTITY,TOTAL_FUEL_CONSUMPTION_MMBTU,ELEC_FUEL_CONSUMPTION_MMBTU,NET_GENERATION_(MEGAWATTHOURS),MMBTU_PER_UNIT_JAN,MMBTU_PER_UNIT_FEB,...,TOTAL_FUEL_CONSUMPTION_QUANTITY %PUR,TOTAL_FUEL_CONSUMPTION_QUANTITY %RFO,TOTAL_FUEL_CONSUMPTION_QUANTITY %SC,TOTAL_FUEL_CONSUMPTION_QUANTITY %SUB,TOTAL_FUEL_CONSUMPTION_QUANTITY %SUN,TOTAL_FUEL_CONSUMPTION_QUANTITY %WAT,TOTAL_FUEL_CONSUMPTION_QUANTITY %WDS,TOTAL_FUEL_CONSUMPTION_QUANTITY %WH,TOTAL_FUEL_CONSUMPTION_QUANTITY %WND,TOTAL_FUEL_CONSUMPTION_QUANTITY %WO
0,3584,H 5,2007,0.0,0.0,109110.0,109110.0,1.103900e+04,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3585,Nolte,2007,0.0,0.0,118993.0,118993.0,1.203900e+04,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,3586,TP 4,2007,0.0,0.0,142735.0,142735.0,1.444100e+04,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,6147,Monticello,2007,11807274.0,11807274.0,171279117.0,171279117.0,1.539900e+07,34.452,34.281,...,0.0,0.0,0.0,0.529492,0.0,0.0,0.0,0.0,0.0,0.0
4,6145,Comanche Peak,2007,0.0,0.0,197597659.0,197597659.0,1.884575e+07,0.000,0.000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
aggEIADict[2015].head()

,ELEC_FUEL_CONSUMPTION_MMBTU,ELEC_FUEL_CONSUMPTION_MMBTU %AB,ELEC_FUEL_CONSUMPTION_MMBTU %BIT,ELEC_FUEL_CONSUMPTION_MMBTU %BLQ,ELEC_FUEL_CONSUMPTION_MMBTU %DFO,ELEC_FUEL_CONSUMPTION_MMBTU %JF,ELEC_FUEL_CONSUMPTION_MMBTU %LFG,ELEC_FUEL_CONSUMPTION_MMBTU %LIG,ELEC_FUEL_CONSUMPTION_MMBTU %MWH,ELEC_FUEL_CONSUMPTION_MMBTU %NG,...,TOT_MMBTU_FEB,TOT_MMBTU_JAN,TOT_MMBTU_JUL,TOT_MMBTU_JUN,TOT_MMBTU_MAR,TOT_MMBTU_MAY,TOT_MMBTU_NOV,TOT_MMBTU_OCT,TOT_MMBTU_SEP,YEAR
0,72974.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,616,9958,3599,12099,6202,9885,4957,1794,3645,2015
1,67999.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,574,9279,3354,11274,5779,9211,4619,1672,3396,2015
2,87001.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,735,11872,4291,14425,7394,11785,5910,2139,4345,2015
3,60253204.0,0.0,0.0,0.0,0.003132,0.0,0.0,0.145169,0.0,0.0,...,...,...,16116830,10240752,0,1480872,5146811,5375612,5227114,2015
4,4059804.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,349648,358283,344795,287042,278416,296580,405538,320529,321004,2015


### Export the EIA 923 data as pickle
Just sending the dictionary to a pickle file for now. At least doing this will save several min of time loading and processing the data in the future.

In [10]:
filename = 'EIA 923.pkl'
path = '../Clean Data'
fullpath = os.path.join(path, filename)

pickle.dump(aggEIADict, open(fullpath, 'wb'))

### Combine all df's from the dict into one df
Concat all dataframes, reset the index, determine the primary fuel type for each facility, filter to only include fossil power plants, and export as a csv

In [11]:
all923 = pd.concat(aggEIADict)

In [12]:
all923.head()

ELEC_FUEL_CONSUMPTION_MMBTU  ELEC_FUEL_CONSUMPTION_MMBTU %AB  \
2007 0                     109110.0                              0.0   
     1                     118993.0                              0.0   
     2                     142735.0                              0.0   
     3                  171279117.0                              0.0   
     4                  197597659.0                              0.0   

        ELEC_FUEL_CONSUMPTION_MMBTU %BIT  ELEC_FUEL_CONSUMPTION_MMBTU %BLQ  \
2007 0                               0.0                               0.0   
     1                               0.0                               0.0   
     2                               0.0                               0.0   
     3                               0.0                               0.0   
     4                               0.0                               0.0   

        ELEC_FUEL_CONSUMPTION_MMBTU %DFO  ELEC_FUEL_CONSUMPTION_MMBTU %JF  \
2007 0                           0.00000                              0.0   
     1                           0.00000                              0.0   
     2                           0.00000                              0.0   
     3                           0.00082                              0.0   
     4                           0.00000                              0.0   

        ELEC_FUEL_CONSUMPTION_MMBTU %LFG  ELEC_FUEL_CONSUMPTION_MMBTU %LIG  \
2007 0                               0.0                          0.000000   
     1                               0.0                          0.000000   
     2                               0.0                          0.000000   
     3                               0.0                          0.403995   
     4                               0.0                          0.000000   

        ELEC_FUEL_CONSUMPTION_MMBTU %MWH  ELEC_FUEL_CONSUMPTION_MMBTU %NG  \
2007 0                               0.0                              0.0   
     1                               0.0                              0.0   
     2                               0.0                              0.0   
     3                               0.0                              0.0   
     4                               0.0                              0.0   

        ...   TOT_MMBTU_FEB  TOT_MMBTU_JAN  TOT_MMBTU_JUL  TOT_MMBTU_JUN  \
2007 0  ...           10344           9710          25442          14517   
     1  ...           11281          10590          27746          15832   
     2  ...           13532          12703          33282          18991   
     3  ...     1.37476e+07    1.39678e+07    1.50848e+07    1.40674e+07   
     4  ...     1.51598e+07    1.83731e+07    1.80729e+07    1.75828e+07   

        TOT_MMBTU_MAR  TOT_MMBTU_MAY  TOT_MMBTU_NOV  TOT_MMBTU_OCT  \
2007 0           1220           7128           2034           2392   
     1           1331           7774           2219           2608   
     2           1596           9325           2661           3129   
     3    1.47215e+07     1.3538e+07    1.45648e+07    1.49483e+07   
     4    9.18252e+06    1.82552e+07    1.77292e+07    1.81813e+07   

        TOT_MMBTU_SEP  YEAR  
2007 0          15284  2007  
     1          16668  2007  
     2          19994  2007  
     3    1.39296e+07  2007  
     4    1.74853e+07  2007  

[5 rows x 174 columns]

In [13]:
all923.reset_index(drop=True, inplace=True)

In [14]:
# Check column numbers to use in the function below
all923.iloc[1,1:27]

ELEC_FUEL_CONSUMPTION_MMBTU %AB     0
ELEC_FUEL_CONSUMPTION_MMBTU %BIT    0
ELEC_FUEL_CONSUMPTION_MMBTU %BLQ    0
ELEC_FUEL_CONSUMPTION_MMBTU %DFO    0
ELEC_FUEL_CONSUMPTION_MMBTU %JF     0
ELEC_FUEL_CONSUMPTION_MMBTU %LFG    0
ELEC_FUEL_CONSUMPTION_MMBTU %LIG    0
ELEC_FUEL_CONSUMPTION_MMBTU %MWH    0
ELEC_FUEL_CONSUMPTION_MMBTU %NG     0
ELEC_FUEL_CONSUMPTION_MMBTU %NUC    0
ELEC_FUEL_CONSUMPTION_MMBTU %OBG    0
ELEC_FUEL_CONSUMPTION_MMBTU %OBL    0
ELEC_FUEL_CONSUMPTION_MMBTU %OBS    0
ELEC_FUEL_CONSUMPTION_MMBTU %OG     0
ELEC_FUEL_CONSUMPTION_MMBTU %OTH    0
ELEC_FUEL_CONSUMPTION_MMBTU %PC     0
ELEC_FUEL_CONSUMPTION_MMBTU %PUR    0
ELEC_FUEL_CONSUMPTION_MMBTU %RFO    0
ELEC_FUEL_CONSUMPTION_MMBTU %SC     0
ELEC_FUEL_CONSUMPTION_MMBTU %SUB    0
ELEC_FUEL_CONSUMPTION_MMBTU %SUN    0
ELEC_FUEL_CONSUMPTION_MMBTU %WAT    1
ELEC_FUEL_CONSUMPTION_MMBTU %WDS    0
ELEC_FUEL_CONSUMPTION_MMBTU %WH     0
ELEC_FUEL_CONSUMPTION_MMBTU %WND    0
ELEC_FUEL_CONSUMPTION_MMBTU %WO     0
Name: 1, dty

In [15]:
def top_fuel(row):
    #Fraction of largest fuel for electric heat input 
    try:
        fuel = row.iloc[1:27].idxmax()[29:]
    except:
        return None
    return fuel

In [16]:
all923['FUEL'] = all923.apply(top_fuel, axis=1)

In [17]:
all923.head()

,ELEC_FUEL_CONSUMPTION_MMBTU,ELEC_FUEL_CONSUMPTION_MMBTU %AB,ELEC_FUEL_CONSUMPTION_MMBTU %BIT,ELEC_FUEL_CONSUMPTION_MMBTU %BLQ,ELEC_FUEL_CONSUMPTION_MMBTU %DFO,ELEC_FUEL_CONSUMPTION_MMBTU %JF,ELEC_FUEL_CONSUMPTION_MMBTU %LFG,ELEC_FUEL_CONSUMPTION_MMBTU %LIG,ELEC_FUEL_CONSUMPTION_MMBTU %MWH,ELEC_FUEL_CONSUMPTION_MMBTU %NG,...,TOT_MMBTU_JAN,TOT_MMBTU_JUL,TOT_MMBTU_JUN,TOT_MMBTU_MAR,TOT_MMBTU_MAY,TOT_MMBTU_NOV,TOT_MMBTU_OCT,TOT_MMBTU_SEP,YEAR,FUEL
0,109110.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,...,9710,25442,14517,1220,7128,2034,2392,15284,2007,WAT
1,118993.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,...,10590,27746,15832,1331,7774,2219,2608,16668,2007,WAT
2,142735.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,...,12703,33282,18991,1596,9325,2661,3129,19994,2007,WAT
3,171279117.0,0.0,0.0,0.0,0.00082,0.0,0.0,0.403995,0.0,0.0,...,1.39678e+07,1.50848e+07,1.40674e+07,1.47215e+07,1.3538e+07,1.45648e+07,1.49483e+07,1.39296e+07,2007,SUB
4,197597659.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,...,1.83731e+07,1.80729e+07,1.75828e+07,9.18252e+06,1.82552e+07,1.77292e+07,1.81813e+07,1.74853e+07,2007,NUC


In [18]:
fossil923 = all923.loc[all923['FUEL'].isin(['DFO', 'LIG', 'NG', 'PC', 'SUB'])]

### Export the EIA 923 data dataframe as csv
Export the dataframe with primary fuel and filtered to only include fossil plants

In [19]:
filename = 'Fossil EIA 923.csv'
path = '../Clean Data'
fullpath = os.path.join(path, filename)
fossil923.to_csv(fullpath)


## Loading the EPA Data, the path may need to be updated...

In [8]:
#Read the EPA files into a dataframe
path2 = os.path.join('EPA air markets')
epaNames = os.listdir(path2)
filePaths = {dn:os.path.join(path2, dn, "*.txt") for dn in epaNames}
filePaths = {dn:glob.glob(val) for dn, val in filePaths.iteritems()}
epaDict = {key:pd.read_csv(fp, index_col = False) for key, val in filePaths.iteritems() for fp in val}

First rename the column name so we can merge on that column, then change the datatype of date to a datetime object

In [9]:
#Rename the column names to remove the leading space.
for key, df in epaDict.iteritems():
    colNames = [name.upper().strip() for name in df.columns]
    colNames[colNames.index("FACILITY ID (ORISPL)")] = "PLANT_ID"
    epaDict[key].columns = colNames
    
#Convert DATE to datetime object
#Add new column DATETIME with both date and hour
for key, df in epaDict.iteritems():
    epaDict[key]["DATE"] = pd.to_datetime(df["DATE"])
    epaDict[key]['DATETIME'] = df['DATE'] + pd.to_timedelta(df['HOUR'], unit='h')

The DataFrames in `epaDict` contain all power plants in Texas. We can filter on `NERC REGION` so that it only includes ERCOT.

In [4]:
set(epaDict['2015 July-Dec'].loc[:,'NERC REGION'])

{nan, 'ERCOT', 'SERC', 'SPP', 'WECC'}

In [10]:
#Boolean filter to only keep ERCOT plants
for key, df in epaDict.iteritems():
    epaDict[key] = df[df["NERC REGION"] == "ERCOT"].reset_index(drop = True)
    

In [6]:
set(epaDict['2015 July-Dec'].loc[:,'NERC REGION'])

{'ERCOT'}

In [25]:
epaDict['2015 July-Dec'].head()

,STATE,FACILITY NAME,PLANT_ID,YEAR,DATE,HOUR,GROSS LOAD (MW),STEAM LOAD (1000LB/HR),SO2 (POUNDS),NOX (POUNDS),CO2 (SHORT TONS),EPA REGION,NERC REGION,COUNTY,HEAT INPUT (MMBTU),FACILITY LATITUDE,FACILITY LONGITUDE,DATETIME
0,TX,Barney M. Davis,4939,2015,2015-07-01,0,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 00:00:00
1,TX,Barney M. Davis,4939,2015,2015-07-01,1,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 01:00:00
2,TX,Barney M. Davis,4939,2015,2015-07-01,2,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 02:00:00
3,TX,Barney M. Davis,4939,2015,2015-07-01,3,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 03:00:00
4,TX,Barney M. Davis,4939,2015,2015-07-01,4,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 04:00:00


### Export EPA data as a series of dataframes
The whole dictionary is too big as a pickle file

In [26]:
# pickle with gzip, from http://stackoverflow.com/questions/18474791/decreasing-the-size-of-cpickle-objects
def save_zipped_pickle(obj, filename, protocol=-1):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)

In [27]:
filename = 'EPA hourly dictionary.pgz'
path = '../Clean Data'
fullpath = os.path.join(path, filename)

save_zipped_pickle(epaDict, fullpath)

In [28]:
df = epaDict['2015 July-Dec']

In [29]:
df.head()

,STATE,FACILITY NAME,PLANT_ID,YEAR,DATE,HOUR,GROSS LOAD (MW),STEAM LOAD (1000LB/HR),SO2 (POUNDS),NOX (POUNDS),CO2 (SHORT TONS),EPA REGION,NERC REGION,COUNTY,HEAT INPUT (MMBTU),FACILITY LATITUDE,FACILITY LONGITUDE,DATETIME
0,TX,Barney M. Davis,4939,2015,2015-07-01,0,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 00:00:00
1,TX,Barney M. Davis,4939,2015,2015-07-01,1,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 01:00:00
2,TX,Barney M. Davis,4939,2015,2015-07-01,2,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 02:00:00
3,TX,Barney M. Davis,4939,2015,2015-07-01,3,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 03:00:00
4,TX,Barney M. Davis,4939,2015,2015-07-01,4,NaN,NaN,NaN,NaN,NaN,6,ERCOT,Nueces,NaN,27.6067,-97.3119,2015-07-01 04:00:00


In [30]:
set(df['PLANT_ID'])

{127,
 298,
 3439,
 3441,
 3443,
 3452,
 3453,
 3460,
 3464,
 3468,
 3469,
 3470,
 3476,
 3477,
 3478,
 3490,
 3491,
 3492,
 3494,
 3497,
 3504,
 3507,
 3548,
 3559,
 3576,
 3601,
 3611,
 3612,
 3628,
 3631,
 4266,
 4937,
 4939,
 6136,
 6139,
 6147,
 6178,
 6179,
 6180,
 6181,
 6183,
 6243,
 6648,
 7030,
 7097,
 7325,
 7900,
 8063,
 52176,
 55015,
 55091,
 55139,
 55153,
 55215,
 55223,
 56350,
 56674}

In [31]:
df_temp = df[df['PLANT_ID'].isin([127, 298, 3439])].fillna(0)

In [32]:
df_temp.head()

,STATE,FACILITY NAME,PLANT_ID,YEAR,DATE,HOUR,GROSS LOAD (MW),STEAM LOAD (1000LB/HR),SO2 (POUNDS),NOX (POUNDS),CO2 (SHORT TONS),EPA REGION,NERC REGION,COUNTY,HEAT INPUT (MMBTU),FACILITY LATITUDE,FACILITY LONGITUDE,DATETIME
79488,TX,Laredo,3439,2015,2015-07-01,0,0.0,0.0,0.0,0.0,0.0,6,ERCOT,Webb,0.0,27.5667,-99.5083,2015-07-01 00:00:00
79489,TX,Laredo,3439,2015,2015-07-01,1,0.0,0.0,0.0,0.0,0.0,6,ERCOT,Webb,0.0,27.5667,-99.5083,2015-07-01 01:00:00
79490,TX,Laredo,3439,2015,2015-07-01,2,0.0,0.0,0.0,0.0,0.0,6,ERCOT,Webb,0.0,27.5667,-99.5083,2015-07-01 02:00:00
79491,TX,Laredo,3439,2015,2015-07-01,3,0.0,0.0,0.0,0.0,0.0,6,ERCOT,Webb,0.0,27.5667,-99.5083,2015-07-01 03:00:00
79492,TX,Laredo,3439,2015,2015-07-01,4,0.0,0.0,0.0,0.0,0.0,6,ERCOT,Webb,0.0,27.5667,-99.5083,2015-07-01 04:00:00


In [ ]:
g = sns.FacetGrid(df_temp, col='PLANT_ID')
g.map(plt.plot, 'datetime', 'GROSS LOAD (MW)')
g.set_xticklabels(rotation=30)

path = os.path.join('..', 'Exploratory visualization', 'Midterm figures', 'Sample hourly load.svg')
plt.savefig(path)

## Finally join the two data sources

Switch to an inner join?

**No need to join. Can keep them as separate databases, since one is hourly data and the other is annual/monthly** Create a clustering dataframe with index of all plant IDs (from the EPA hourly data), add columns with variables. Calculate the inputs in separate dataframes - example is to calculate ramp rate values in the EPA hourly data, then put the results in the clustering dataframe.

In [34]:
#Join the two data sources on PLANT_ID
fullData = {key:df.merge(aggEIADict[df["YEAR"][0]], on="PLANT_ID") for key, df in epaDict.iteritems()}

In [35]:
fullData[fullData.keys()[0]].head()

,STATE,FACILITY NAME,PLANT_ID,YEAR_x,DATE,HOUR,GROSS LOAD (MW),STEAM LOAD (1000LB/HR),SO2 (POUNDS),NOX (POUNDS),...,TOT_MMBTU_FEB,TOT_MMBTU_JAN,TOT_MMBTU_JUL,TOT_MMBTU_JUN,TOT_MMBTU_MAR,TOT_MMBTU_MAY,TOT_MMBTU_NOV,TOT_MMBTU_OCT,TOT_MMBTU_SEP,YEAR_y
0,TX,Barney M. Davis,4939,2012,2012-01-01,0,NaN,NaN,NaN,NaN,...,1.49482e+06,1.28956e+06,1.98845e+06,2.02212e+06,1.93844e+06,2.06786e+06,905868,1.51242e+06,1.88822e+06,2012
1,TX,Barney M. Davis,4939,2012,2012-01-01,1,NaN,NaN,NaN,NaN,...,1.49482e+06,1.28956e+06,1.98845e+06,2.02212e+06,1.93844e+06,2.06786e+06,905868,1.51242e+06,1.88822e+06,2012
2,TX,Barney M. Davis,4939,2012,2012-01-01,2,NaN,NaN,NaN,NaN,...,1.49482e+06,1.28956e+06,1.98845e+06,2.02212e+06,1.93844e+06,2.06786e+06,905868,1.51242e+06,1.88822e+06,2012
3,TX,Barney M. Davis,4939,2012,2012-01-01,3,NaN,NaN,NaN,NaN,...,1.49482e+06,1.28956e+06,1.98845e+06,2.02212e+06,1.93844e+06,2.06786e+06,905868,1.51242e+06,1.88822e+06,2012
4,TX,Barney M. Davis,4939,2012,2012-01-01,4,NaN,NaN,NaN,NaN,...,1.49482e+06,1.28956e+06,1.98845e+06,2.02212e+06,1.93844e+06,2.06786e+06,905868,1.51242e+06,1.88822e+06,2012


BIT, SUB, LIG, NG, DFO, RFO

In [36]:
[x for x in fullData[fullData.keys()[0]].columns]

['STATE',
 'FACILITY NAME',
 'PLANT_ID',
 'YEAR_x',
 'DATE',
 'HOUR',
 'GROSS LOAD (MW)',
 'STEAM LOAD (1000LB/HR)',
 'SO2 (POUNDS)',
 'NOX (POUNDS)',
 'CO2 (SHORT TONS)',
 'HEAT INPUT (MMBTU)',
 'EPA REGION',
 'NERC REGION',
 'COUNTY',
 'FACILITY LATITUDE',
 'FACILITY LONGITUDE',
 'DATETIME',
 u'ELEC_FUEL_CONSUMPTION_MMBTU',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %AB',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %BIT',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %BLQ',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %DFO',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %JF',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %LFG',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %LIG',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %MWH',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %NG',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %NUC',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %OBG',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %OBL',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %OBS',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %OG',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %OTH',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %PC',
 u'ELEC_FUEL_CONSUMPTION_MMBTU %PUR',
 u'ELEC_FUEL_CONSUMPTION_MMBTU 

## Loading EIA 860 Data


In [11]:
# Iterate through the directory to find all the files to import
path = os.path.join('EIA Data', '860-No_Header')
full_path = os.path.join(path, '*.*')

eia860Names = os.listdir(path)

# Rename the keys for easier merging later
fileName860Map = {  'GenY07.xls':2007,
                    'GenY08.xls':2008,
                    'GeneratorY09.xls':2009,
                    'GeneratorsY2010.xls':2010,
                    'GeneratorY2011.xlsx':2011,
                    'GeneratorY2012.xlsx':2012,
                    '3_1_Generator_Y2013.xlsx':2013,
                    '3_1_Generator_Y2014.xlsx':2014,
                    '3_1_Generator_Y2015.xlsx':2015}

#Load the files into data frames, one df per file
eia860Dict = {fileName860Map[fn]:pd.read_excel(os.path.join(path, fn)) for fn in eia860Names}  

In [12]:
#Dict of values to replace to standardize column names across all dataframes
renameDict = {  "PLNTCODE":"PLANT_ID",
                "PLANT_CODE":"PLANT_ID",
                "Plant Code":"PLANT_ID",
                "NAMEPLATE":"NAMEPLATE_CAPACITY(MW)",
                "Nameplate Capacity (MW)":"NAMEPLATE_CAPACITY(MW)"}

#Replace the column name
def rename860(col):
    for old, new in renameDict.iteritems():
        col = col.replace(old, new)
    return col

#Iterate through each column name of each dataframe to standardize and select columns 'PLANT_ID', 'NAMEPLATE_CAPACITY(MW)'
for key, df in eia860Dict.iteritems():
    colNames = [rename860(col) for col in df.columns]
    eia860Dict[key].columns = colNames
    eia860Dict[key] = eia860Dict[key][["PLANT_ID", "NAMEPLATE_CAPACITY(MW)"]]

In [39]:
# verify the tables
for key, df in eia860Dict.iteritems():
    print key, df.columns, len(df)

2007 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 19060
2008 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 19558
2009 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 17876
2010 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 18150
2011 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 18530
2012 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 19023
2013 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 19243
2014 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 19745
2015 Index([u'PLANT_ID', u'NAMEPLATE_CAPACITY(MW)'], dtype='object') 20068


In [13]:
# Iterate through each dataframe to perform aggregation by PLANT_ID
for key, df in eia860Dict.iteritems():
    gb = df.groupby(by='PLANT_ID').apply(lambda x: x['NAMEPLATE_CAPACITY(MW)'].sum())
    eia860Dict[key]['NAMEPLATE_CAPACITY(MW)'] = eia860Dict[key].PLANT_ID.apply(gb.get_value)
    eia860Dict[key] = eia860Dict[key].drop_duplicates(subset=['PLANT_ID', 'NAMEPLATE_CAPACITY(MW)'])
    eia860Dict[key] = eia860Dict[key].sort_values(by='PLANT_ID').reset_index(drop=True)

### Export EIA 860 data

In [41]:
filename = 'EIA 860.pkl'
path = '../Clean Data'
fullpath = os.path.join(path, filename)

pickle.dump(eia860Dict, open(fullpath, 'wb'))

## Creating Final DataFrame for Clustering Algorithm:
#### clusterDict {year : cluster_DF}

1. For each PLANT_ID in aggEIADict, fetch the corresponding aggregated NAMEPLATE_CAPACITY(MW)

In [34]:
clusterDict = dict()
for key, df in eia860Dict.iteritems():
    clusterDict[key] = pd.merge(aggEIADict[key], eia860Dict[key], how='left', on='PLANT_ID')[['PLANT_ID', 'NAMEPLATE_CAPACITY(MW)']]
    clusterDict[key].rename(columns={'NAMEPLATE_CAPACITY(MW)': 'capacity', 'PLANT_ID': 'plant_id'}, inplace=True)

In [35]:
# verify for no loss of data
for key, df in eia860Dict.iteritems():
    print key, len(clusterDict[key]), len(aggEIADict[key])

2007 216 216
2008 234 234
2009 245 245
2010 253 253
2011 257 257
2012 274 274
2013 282 282
2014 291 291
2015 312 312


In [36]:
clusterDict[2015].head()

,plant_id,capacity
0,3584,2.4
1,3585,2.4
2,3586,2.4
3,6147,1980.0
4,58372,135.4


Function to get fuel type

In [16]:
fuel_cols = [col for col in aggEIADict[2008].columns if 'ELEC_FUEL_CONSUMPTION_MMBTU %' in col]

def top_fuel(row):
    #Fraction of largest fuel for electric heat input 
    try:
        fuel = row.idxmax()[29:]
    except:
        return None
    return fuel

# clusterDict[2008]['fuel'] = aggEIADict[2008][fuel_cols].apply(top_fuel, axis=1)

Calculate Capacity factor, Efficiency, Fuel type

In [37]:
for key, df in clusterDict.iteritems():
    clusterDict[key]['year'] = key
    clusterDict[key]['capacity_factor'] = aggEIADict[key]['NET_GENERATION_(MEGAWATTHOURS)'] / (8670*clusterDict[key]['capacity'])
    clusterDict[key]['efficiency'] = (aggEIADict[key]['NET_GENERATION_(MEGAWATTHOURS)']*3.412)/(1.0*aggEIADict[key]['ELEC_FUEL_CONSUMPTION_MMBTU'])
    clusterDict[key]['fuel_type'] = aggEIADict[key][fuel_cols].apply(top_fuel, axis=1)
    clusterDict[key] = clusterDict[key][clusterDict[key]['fuel_type'].isin(['SUB', 
                                                                            'LIG', 
                                                                            'DFO',
                                                                            'NG', 
                                                                            'RFO'])]

Merge all epa files in one df

In [29]:
columns = ['PLANT_ID', 'YEAR', 'DATE', 'HOUR', 'GROSS LOAD (MW)']
counter = 0
for key, df in epaDict.iteritems():
    if counter == 0:
        result = epaDict[key][columns]
        counter = 1
    else:
        result = result.append(epaDict[key][columns], ignore_index=True)
        
# Change nan to 0
result.fillna(0, inplace=True)

In [49]:
result.describe()

,PLANT_ID,YEAR,HOUR,GROSS LOAD (MW)
count,4.511880e+06,4.511880e+06,4.511880e+06,4.511880e+06
mean,1.006244e+04,2.010944e+03,1.150000e+01,3.000693e+02
std,1.616199e+04,2.613026e+00,6.922187e+00,5.057269e+02
min,1.270000e+02,2.007000e+03,0.000000e+00,0.000000e+00
25%,3.477000e+03,2.009000e+03,5.750000e+00,0.000000e+00
50%,3.611000e+03,2.011000e+03,1.150000e+01,4.100000e+01
75%,6.183000e+03,2.013000e+03,1.725000e+01,4.180000e+02
max,5.667400e+04,2.015000e+03,2.300000e+01,3.871000e+03


Function to calculate the ramp rate for every hour

In [19]:
def plant_gen_delta(df):
    """
    For every plant in the input df, calculate the change in gross load (MW)
    from the previous hour.
    
    input:
        df: dataframe of EPA clean air markets data
    return:
        df: concatanated list of dataframes
    """
    df_list = []
    for plant in df['PLANT_ID'].unique():
        temp = df.loc[df['PLANT_ID'] == plant,:]
        gen_change = temp.loc[:,'GROSS LOAD (MW)'].values - temp.loc[:,'GROSS LOAD (MW)'].shift(1).values
        temp.loc[:,'Gen Change'] = gen_change
        df_list.append(temp)
    return pd.concat(df_list)

In [30]:
ramp_df = plant_gen_delta(result)

In [50]:
ramp_df.describe()

,PLANT_ID,YEAR,HOUR,GROSS LOAD (MW),Gen Change
count,4.511880e+06,4.511880e+06,4.511880e+06,4.511880e+06,4.511811e+06
mean,1.006244e+04,2.010944e+03,1.150000e+01,3.000693e+02,8.391309e-04
std,1.616199e+04,2.613026e+00,6.922187e+00,5.057269e+02,5.223558e+01
min,1.270000e+02,2.007000e+03,0.000000e+00,0.000000e+00,-1.996000e+03
25%,3.477000e+03,2.009000e+03,5.750000e+00,0.000000e+00,0.000000e+00
50%,3.611000e+03,2.011000e+03,1.150000e+01,4.100000e+01,0.000000e+00
75%,6.183000e+03,2.013000e+03,1.725000e+01,4.180000e+02,0.000000e+00
max,5.667400e+04,2.015000e+03,2.300000e+01,3.871000e+03,2.014000e+03


Get the max ramp rate for every plant for each year

In [21]:
cols = ['PLANT_ID', 'YEAR', 'Gen Change']

ramp_rate_list = []
for year in ramp_df['YEAR'].unique():
    for plant in ramp_df.loc[ramp_df['YEAR']==year,'PLANT_ID'].unique():
        # 95th percentile ramp rate per plant per year
        ramp_95 = ramp_df.loc[(ramp_df['PLANT_ID']== plant) & 
                              (ramp_df['YEAR']==year),'Gen Change'].quantile(0.95, interpolation='nearest')
        ramp_rate_list.append([plant, year, ramp_95])

In [22]:
ramp_rate_df = pd.DataFrame(ramp_rate_list, columns=['plant_id', 'year', 'ramp_rate'])

In [23]:
ramp_rate_df.describe()

,plant_id,year,ramp_rate
count,521.000000,521.000000,521.000000
mean,10039.737044,2010.934741,52.230326
std,16157.049504,2.615700,57.270081
min,127.000000,2007.000000,0.000000
25%,3477.000000,2009.000000,6.000000
50%,3611.000000,2011.000000,34.000000
75%,6181.000000,2013.000000,83.000000
max,56674.000000,2015.000000,373.000000


In [40]:
ramp_rate_df[ramp_rate_df.plant_id==10243]

,plant_id,year,ramp_rate


In [42]:
ramp_df[ramp_df.PLANT_ID==10243]

,PLANT_ID,YEAR,DATE,HOUR,GROSS LOAD (MW),Gen Change


In [38]:
for key, df in clusterDict.iteritems():
    clusterDict[key] = pd.merge(clusterDict[key], ramp_rate_df, how='left', on=['plant_id', 'year'])

In [39]:
clusterDict[2010].head()

,plant_id,capacity,year,capacity_factor,efficiency,fuel_type,ramp_rate
0,6147,1980.0,2010,0.783340,0.306713,SUB,184.0
1,3601,639.0,2010,0.158669,0.294749,NG,67.0
2,10243,44.2,2010,0.570003,0.658194,NG,NaN
3,10261,25.7,2010,0.843285,0.721433,NG,NaN
4,55320,746.0,2010,0.772931,0.598151,NG,NaN


In [51]:
# Check plants larger than 25MW, which is the lower limit for EPA
clusterDict[2010][clusterDict[2010].capacity >=25].describe()

,plant_id,capacity,year,capacity_factor,efficiency,ramp_rate
count,121.000000,121.000000,121.0,121.000000,121.000000,50.000000
mean,28787.338843,677.142975,2010.0,0.370894,0.408985,60.500000
std,24501.935749,561.024245,0.0,0.272095,0.147305,59.216397
min,127.000000,25.700000,2010.0,0.000542,0.165060,0.000000
25%,3630.000000,250.000000,2010.0,0.086994,0.297452,16.500000
50%,10692.000000,595.000000,2010.0,0.394855,0.369894,52.500000
75%,55132.000000,927.500000,2010.0,0.583774,0.463560,93.000000
max,57504.000000,4008.400000,2010.0,0.922401,0.810491,328.000000


In [47]:
for key in clusterDict.keys():
    print key, clusterDict[key].plant_id.count(), clusterDict[key].ramp_rate.count()

2007 140 52
2008 136 52
2009 135 49
2010 136 50
2011 133 50
2012 134 50
2013 136 50
2014 137 51
2015 136 52


#### ERROR Record - Efficiency!!!

In [143]:
print clusterDict[2007].loc[156].to_string()

plant_id                  3466
capacity                2314.5
year                      2007
capacity_factor   -0.000217325
efficiency                -inf
fuel_type                   AB
ramp_rate                  NaN


Save dict to csv

In [141]:
# re-arrange column order
columns = ['year', 'plant_id', 'capacity', 'capacity_factor', 'efficiency', 'ramp_rate', 'fuel_type']

filename = 'Cluster_Data.csv'
path = '../Clean Data'
fullpath = os.path.join(path, filename)

counter = 0
for key, df in clusterDict.iteritems():
    # create the csv file
    if counter == 0:
        df[columns].sort_values(by='plant_id').to_csv(fullpath, sep=',',  index = False)
        counter += 1
    # append to existing csv file
    else:
        df[columns].sort_values(by='plant_id').to_csv(fullpath, sep=',',  index = False, header=False, mode = 'a')

## Calculating ramp rate
Calculate rate of change over 1,2,3 hours for positive change. 

# Assumptions
1. Plant capacity changes at the start of the year and is constant for the entire year
2. Same for ramp rate - no changes over the course of the year